In [20]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
train_df = pd.read_csv('train.csv')

C:\Users\hp\AppData\Local\Temp\ipykernel_16308\2615139025.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('train.csv')


In [22]:
# Remove rows with missing values and duplicates in training data
train_df.dropna(inplace=True)
train_df.drop_duplicates(inplace=True)

In [23]:
# Ensure only relevant columns are kept
train_df = train_df[['label', 'comment']]

In [24]:
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)
    
    return text

train_df['cleaned_comment'] = train_df['comment'].apply(clean_text)

In [25]:
# Balancing the training dataset
count_1s = train_df['label'].value_counts()[1]
count_0s = train_df['label'].value_counts()[0]
diff = abs(count_1s - count_0s)

In [27]:
print(count_1s)

41361


In [29]:
print(count_0s)

55803


In [30]:
if count_1s > count_0s:
    # Remove extra 1s
    data_1s = train_df[train_df['label'] == 1]
    data_0s = train_df[train_df['label'] == 0]
    data_1s_balanced = data_1s.sample(n=count_0s)
    balanced_train_df = pd.concat([data_1s_balanced, data_0s])
else:
    # Remove extra 0s
    data_1s = train_df[train_df['label'] == 1]
    data_0s = train_df[train_df['label'] == 0]
    data_0s_balanced = data_0s.sample(n=count_1s)
    balanced_train_df = pd.concat([data_1s, data_0s_balanced])

In [31]:
# Save the cleaned and balanced datasets
balanced_train_df.to_csv('cleaned_balanced_train_data.csv', index=False)

In [32]:
# Display the first few rows of the cleaned and balanced training data
print("Cleaned and Balanced Training Data:")
print(balanced_train_df.head())

Cleaned and Balanced Training Data:
    label                                            comment  \
30    1.0                But they'll have all those reviews!   
40    1.0  wow it is totally unreasonable to assume that ...   
41    1.0  Ho ho ho... But Melania said that there is no ...   
59    1.0  I can't wait until @potus starts a twitter war...   
62    1.0  gotta love the teachers who give exams on the ...   

                                      cleaned_comment  
30                                     theyll reviews  
40  wow totally unreasonable assume agency covered...  
41  ho ho ho melania said way could happened didnt...  
59     cant wait potus starts twitter war morning joe  
62       gotta love teachers give exams day halloween  


In [33]:
# Display the balanced class distribution in training data
print("\nBalanced Dataset:")
print(balanced_train_df['label'].value_counts())


Balanced Dataset:
label
1.0    41361
0.0    41361
Name: count, dtype: int64
